In [1]:
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
from random import sample
import os

import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

сделать матрицу

In [2]:
os.listdir('data')    

['Dewas_NDVI.csv',
 'Dewas_points.csv',
 'Kaithal_NDVI.csv',
 'Kaithal_points.csv',
 'Karnal_NDVI.csv',
 'Karnal_points.csv']

In [3]:
data_points = pd.read_csv('data/Kaithal_points.csv')

In [4]:
data_ndvi = pd.read_csv('data/Kaithal_NDVI.csv')

In [5]:
data = data_points.merge(data_ndvi, left_on='gfid', right_on='gfid')

In [6]:
data['month'] = data['date'].apply(pd.to_datetime).dt.month
data['day'] = data['date'].apply(pd.to_datetime).dt.day

In [7]:
data_ndvi.head()

,gfid,datenum,date,ndvi
0,52001,0,2020-10-20,0.238
1,52001,1,2020-10-21,0.235
2,52001,2,2020-10-22,0.233
3,52001,3,2020-10-23,0.231
4,52001,4,2020-10-24,0.228


In [8]:
i = 1
ids_not_full = []
for id_ in data_points['gfid'].values:
    shp = data_ndvi[data_ndvi['gfid'] == id_].shape[0]
    if shp <= 160:
        #print(id_, ' ', shp)
        i += 1
        ids_not_full += [id_]
#print(i)

In [9]:
data2 = data[~data['gfid'].isin(ids_not_full)]

In [10]:
gfids = data2['gfid'].unique()

In [11]:
from numpy.random import choice

In [12]:
import numpy as np

In [13]:
np.random.seed(42)

In [14]:
np.random.shuffle(gfids)

In [15]:
train_ids = gfids[:150]
test_ids = gfids[150:]

In [39]:
import json
indices = {'Kaithal_train_ids':train_ids.tolist(),
 'Kaithal_test_ids':test_ids.tolist()}
with open('Kaithal_indices.json', 'w') as f:
    json.dump(indices, f)

In [16]:
data2.head()

,gfid,state,district,village,lon,lat,wheat,datenum,date,ndvi,month,day
0,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,0,2020-10-20,0.238,10,20
1,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,1,2020-10-21,0.235,10,21
2,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,2,2020-10-22,0.233,10,22
3,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,3,2020-10-23,0.231,10,23
4,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,4,2020-10-24,0.228,10,24


In [17]:
data2 = data2.dropna()

In [18]:
data2.shape

(38417, 12)

In [19]:
data2.shape

(38417, 12)

In [21]:
for i in (3, 5, 7, 9):
    data2[f'ndvi_{i}'] = data2['ndvi'].rolling(i).mean()

In [22]:
datasets_train = []
datasets_test = []
y_train = []
y_test = []
for _ in gfids:
    rolling_data = []
    rolling_data.append(data2[data2['gfid']==_]['ndvi'].iloc[4:])
    for i in range(2, 11):
        rolling_data.append(data2[data2['gfid']==_]['ndvi'].rolling(i).mean())
    data_1_rolling = pd.concat(rolling_data, axis=1)
    data_1_rolling = data_1_rolling.dropna()
    data_1_rolling.columns = ['ndvi'] + [f'ndvi_{i}' for i in range(2,11)]
    if _ in train_ids:
        datasets_train.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_train += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]
    else:
        datasets_test.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_test += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]

In [23]:
data_train = torch.stack([torch.Tensor(dataset) for dataset in datasets_train])
data_test = torch.stack([torch.Tensor(dataset) for dataset in datasets_test])

In [24]:
def calc_out(h_in, w_in, ker, pad=0, stride=1):
    h_out = (h_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    w_out = (w_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    return h_out, w_out

In [25]:
class AlexNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes  #number of classes
        
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=30, kernel_size=2, stride=2, padding=1),  # (b x 30 x 6 x 6)
            nn.ReLU(),
            nn.Conv2d(30, 64, 2, padding=0, stride=1),  # (b x 64 x 5 x 5)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),  # (b x 64 x 4 x 4)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1, inplace=True),
            nn.Linear(in_features=(64 * 4 * 4), out_features=256),
            nn.ReLU(),
            #nn.Dropout(p=0.5, inplace=True),
            #nn.Linear(in_features=256, out_features=256),
            #nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes),
            nn.Sigmoid()
        )
        #self.init_bias()

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[0].bias, 1)
        nn.init.constant_(self.net[2].bias, 1)
        #nn.init.constant_(self.net[4].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = x.view(-1, 64 * 4 * 4)  # reduce the dimensions for linear layer input
        return self.classifier(x)

In [26]:
alexnet = AlexNet(num_classes=2)

In [27]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=alexnet.parameters(), lr=0.0001)#1, weight_decay=0.5)

In [28]:
torch.autograd.set_detect_anomaly(True)

In [29]:
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

In [30]:
X = data_train
y = y_train_tensors

In [31]:
n_epochs = 100  # or whatever
batch_size = 30  # or whatever

for epoch in range(n_epochs):

    # X is a torch Variable
    permutation = torch.randperm(X.size()[0])

    for i in range(0, X.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X[indices], y[indices]

        outputs = alexnet.forward(batch_x) #forward pass
        #optimizer.zero_grad() #caluclate the gradient, manually setting to 0

        # obtain the loss function
        loss = criterion(outputs, batch_y)
        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.69343
Epoch: 1, loss: 0.69391
Epoch: 2, loss: 0.69149
Epoch: 3, loss: 0.69404
Epoch: 4, loss: 0.69536
Epoch: 5, loss: 0.69577
Epoch: 6, loss: 0.69751
Epoch: 7, loss: 0.69302
Epoch: 8, loss: 0.68663
Epoch: 9, loss: 0.68652
Epoch: 10, loss: 0.69114
Epoch: 11, loss: 0.68500
Epoch: 12, loss: 0.68643
Epoch: 13, loss: 0.68827
Epoch: 14, loss: 0.68137
Epoch: 15, loss: 0.68617
Epoch: 16, loss: 0.69530
Epoch: 17, loss: 0.68360
Epoch: 18, loss: 0.69152
Epoch: 19, loss: 0.67578
Epoch: 20, loss: 0.67082
Epoch: 21, loss: 0.68276
Epoch: 22, loss: 0.68323
Epoch: 23, loss: 0.67781
Epoch: 24, loss: 0.67063
Epoch: 25, loss: 0.67377
Epoch: 26, loss: 0.68225
Epoch: 27, loss: 0.67679
Epoch: 28, loss: 0.67655
Epoch: 29, loss: 0.68824
Epoch: 30, loss: 0.67299
Epoch: 31, loss: 0.65655
Epoch: 32, loss: 0.66256
Epoch: 33, loss: 0.65610
Epoch: 34, loss: 0.68420
Epoch: 35, loss: 0.65398
Epoch: 36, loss: 0.67358
Epoch: 37, loss: 0.65537
Epoch: 38, loss: 0.63511
Epoch: 39, loss: 0.63447
Epoch: 40,

In [32]:
X_test_tensors = data_test

In [33]:
test_predict = alexnet(X_test_tensors)#forward pass 

In [34]:
test_predict

tensor([[0.8086, 0.1918],
        [0.2752, 0.6885],
        [0.5602, 0.4339],
        [0.2033, 0.7775],
        [0.1178, 0.8666],
        [0.3628, 0.6180],
        [0.8010, 0.1971],
        [0.2090, 0.7802],
        [0.9333, 0.0699],
        [0.7175, 0.2854],
        [0.0970, 0.8874],
        [0.2687, 0.7113],
        [0.7674, 0.2406],
        [0.1976, 0.7889],
        [0.4129, 0.5608],
        [0.9100, 0.0879],
        [0.2405, 0.7482],
        [0.9354, 0.0665],
        [0.3131, 0.6735],
        [0.7136, 0.2894],
        [0.4766, 0.4993],
        [0.4341, 0.5472],
        [0.2099, 0.7765],
        [0.1802, 0.7935],
        [0.9449, 0.0558],
        [0.9581, 0.0435],
        [0.0762, 0.9138],
        [0.4748, 0.5073],
        [0.2847, 0.6940],
        [0.8700, 0.1276],
        [0.2538, 0.7219],
        [0.1843, 0.8076],
        [0.3149, 0.6816],
        [0.1468, 0.8378],
        [0.9344, 0.0672],
        [0.7806, 0.2292],
        [0.5857, 0.3903],
        [0.9007, 0.1034],
        [0.6

In [35]:
test_predict.argmax(dim=1)

tensor([0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1])

In [36]:
# 100 эпох, батч 30
print(classification_report(y_test_tensors, torch.argmax(test_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.56      0.56      0.56        18
           1       0.67      0.67      0.67        24

    accuracy                           0.62        42
   macro avg       0.61      0.61      0.61        42
weighted avg       0.62      0.62      0.62        42



In [37]:
train_predict = alexnet(X)#forward pass 

In [38]:
# 100 эпох, батч 30
print(classification_report(y_train_tensors, torch.argmax(train_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.75      0.56      0.64        70
           1       0.68      0.84      0.75        80

    accuracy                           0.71       150
   macro avg       0.72      0.70      0.70       150
weighted avg       0.71      0.71      0.70       150

